In [1]:
!pip install datasets transformers accelerate bitsandbytes huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification

import torch
from torch.utils.data import DataLoader

from sklearn.metrics import accuracy_score, precision_recall_fscore_support

from transformers import BitsAndBytesConfig

In [3]:
data = load_dataset("agraj07/phishing_dataset")
data

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/515 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/130k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/28.3k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.9k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2800 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/600 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/600 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 2800
    })
    validation: Dataset({
        features: ['text', 'labels'],
        num_rows: 600
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 600
    })
})

In [4]:
device = torch.device('cuda')

In [10]:
from transformers import DistilBertTokenizerFast

tokenizer = DistilBertTokenizerFast.from_pretrained("bert-base-uncased")

model_id = "agraj07/bert-phising-classifier-student"
#tokenizer = DistilBertTokenizerFast.from_pretrained("agraj07/bert-phising-classifier-student")
model = AutoModelForSequenceClassification.from_pretrained(model_id).to(device)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'DistilBertTokenizerFast'.


model.safetensors:   0%|          | 0.00/211M [00:00<?, ?B/s]

In [11]:
def preprocess(examples):
  return tokenizer(examples['text'], padding='max_length', truncation=True)

tokenized_data = data.map(preprocess, batched=True)
tokenized_data.set_format(type='torch',columns=['input_ids', 'attention_mask','labels'])

Map:   0%|          | 0/2800 [00:00<?, ? examples/s]

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

In [12]:
def evaluate_model(model, dataloader, device):
    model.eval()  # Set model to evaluation mode
    all_preds = []
    all_labels = []

    # Disable gradient calculations
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            # Forward pass to get logits
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            # Get predictions
            preds = torch.argmax(logits, dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.cpu().numpy())

    # Calculate evaluation metrics
    accuracy = accuracy_score(all_labels, all_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='binary')

    return accuracy, precision, recall, f1

In [13]:
#Evaluation (pre-quantization)
validation_dataloader = DataLoader(tokenized_data['validation'],batch_size=128)
pre_accuracy, pre_precision, pre_recall, pre_f1 = evaluate_model(model, validation_dataloader, device)
print(f"Pre-quantization - Accuracy: {pre_accuracy:.4f}, Precision: {pre_precision:.4f}, Recall: {pre_recall:.4f}, F1: {pre_f1:.4f}")

Pre-quantization - Accuracy: 0.9283, Precision: 0.9512, Recall: 0.9040, F1: 0.9270


In [14]:
#quantizing the model

quantize_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

quantized_model = AutoModelForSequenceClassification.from_pretrained(model_id, quantization_config=quantize_config).to(device)

`low_cpu_mem_usage` was None, now default to True since model is quantized.


In [15]:
#Evaluation (post-quantization)

post_accuracy, post_precision, post_recall, post_f1 = evaluate_model(quantized_model, validation_dataloader, device)
print(f"Post-quantization - Accuracy: {post_accuracy:.4f}, Precision: {post_precision:.4f}, Recall: {post_recall:.4f}, F1: {post_f1:.4f}")

Post-quantization - Accuracy: 0.9333, Precision: 0.9486, Recall: 0.9172, F1: 0.9327


In [16]:
from huggingface_hub import notebook_login

notebook_login()

In [17]:
quantized_model.push_to_hub("agraj07/bert-phishing-classifier_student_4bit")

model.safetensors:   0%|          | 0.00/62.7M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/agraj07/bert-phishing-classifier_student_4bit/commit/c6c93710b44ae6b9183c097837e34af6b02b03aa', commit_message='Upload DistilBertForSequenceClassification', commit_description='', oid='c6c93710b44ae6b9183c097837e34af6b02b03aa', pr_url=None, repo_url=RepoUrl('https://huggingface.co/agraj07/bert-phishing-classifier_student_4bit', endpoint='https://huggingface.co', repo_type='model', repo_id='agraj07/bert-phishing-classifier_student_4bit'), pr_revision=None, pr_num=None)